In [2]:
import pandas as pd
df = pd.read_csv('Diabetes_Drug-Drug_Interactions.csv')
df = df.rename({"Drug": "Source", "Not Advised With (Specific Drug, no need to list drugs if they belong to a class that's in the previous column)": "Single Drug", "Not Advised With (Class of Drug, put SINGULAR!)": "Adverse Class"}, axis=1)
df

,Class,Source,Alternate Names,Adverse Class,Single Drug,Classification of Risk,Mechanism,Adverse Effect,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Biguanide,Metformin,"Glucophage, Fortamet, Glutetza, Riomet",Sulfonylureas,glipizide,Severe,NaN,Increased Risk of Cardiovascular Disease or Al...,NaN,NaN,NaN,NaN
1,Thiazolidinediones (TZD),pioglitazone,Actos,Sulfonylureas (couldn't find a drug advised ag...,secretagogue (couldn't find a drug advised aga...,Minor,positive mechanism,"very slight hypoglycemia effects, but mostly p...",NaN,NaN,NaN,"Classification of Risk: Mild, Moderate, Severe"
2,Thiazolidinediones (TZD),rosiglitazone,Avandia,Sulfonylureas (couldn't find a drug advised ag...,NaN,Minor,positive mechanism,"very slight hypoglycemia effects, but mostly p...",NaN,NaN,NaN,NaN
3,GLP1 mimetics,albiglutide,NaN,Meglitinides,glipizide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
4,GLP1 mimetics,dulaglutide,NaN,Meglitinides,repaglinide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
5,GLP1 mimetics,exenatide,NaN,Meglitinides,glipizide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
6,GLP1 mimetics,liraglutide,victoza,Meglitinides,repaglinide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
7,GLP1 mimetics,lixisenatide,"soliqua, insulin glargine",Meglitinides,glipizide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
8,GLP1 mimetics,semaglutide,ozempic,Meglitinides,repaglinide,Moderate,GLP 1 receptor agonists stimulate insulin secr...,hypoglycemia risks,NaN,NaN,NaN,NaN
9,DPP4,alogliptin,"Nesina, Vipidia",Meglitinides,repaglinide,Moderate,NaN,hypoglycemia risks,NaN,NaN,NaN,NaN


In [3]:
"""First write out straightforward source - target - label triplets"""
df_singles = df[df['Single Drug'].notna()] # slice dataframe to just see
# need to map from "Adverse Class" to "Class" column and fetch those drugs 
df_singles = df_singles[['Source', 'Single Drug', 'Classification of Risk']]
df_singles = df_singles.rename({"Source": 'source', 'Single Drug': 'target', 'Classification of Risk': 'label'}, axis=1)
df_singles.to_csv('./ddis_from_spreadsheet.csv', index=False)

In [4]:
def add_list(drug_class, drug_class_mappings):
    try:
        return drug_class_mappings[drug_class]
    except:
        return None

In [5]:
"""1. Look for individual drugs that have an Adverse Class"""
drugs_with_adverse_class = df[df['Adverse Class'].notna()]

"""2. Get mapping of class:drugs in that class"""
classes = set(df['Class'])
# need to get the union of adverse classes and regular classes
adv_classes = set(df['Adverse Class'])
classes = classes.union(adv_classes)

class_mapping = {}
for class_ in classes:
    drugs_in_class = set(df[df['Class'] == class_]['Source'])
    class_mapping[class_] = drugs_in_class
# print(class_mapping)

"""2. For every drug with an Adverse Class, find the drugs that correspond to that class and create edges."""
df_new = df.copy()
df_new['Drugs in Adverse Class'] = drugs_with_adverse_class['Adverse Class'].apply(add_list, drug_class_mappings = class_mapping)
df_new = df_new.explode('Drugs in Adverse Class')
df_classes_slice = df_new[['Source', 'Drugs in Adverse Class', 'Classification of Risk']]

# now remove triples with no Single Drug
df_classes_rm = df_classes_slice.dropna(subset=['Drugs in Adverse Class'])
df_classes_rm.to_csv('./ddis_from_spreadsheet.csv', mode='a', header=False, index=False)

In [6]:
class_mapping

{'Thiazolidinediones (TZD)': {'pioglitazone', 'rosiglitazone'},
 'DPP4': {'alogliptin', 'linagliptin', 'saxagliptin', 'sitagliptin'},
 'H2-antagonoists': set(),
 'Insulin': set(),
 'Alpha-glucosidase inhibitor': {'acarbose', 'miglitol'},
 "Sulfonylureas (couldn't find a drug advised against, so found a drug worked well with) + Metformin": set(),
 'Fibrates': set(),
 'SGLT2 inhibitors': {'canagliflozin',
  'dapagliflozin',
  'empagliflozin',
  'ertugliflozin'},
 nan: set(),
 'Meglitinides': {'nateglinide', 'repaglinide'},
 'Biguanide': {'Metformin'},
 'Sulfonylureas': {'chlorpropamide',
  'gliclazide',
  'glimepiride',
  'glipizide',
  'glyburide',
  'tolazamide',
  'tolbutamide'},
 'Anticholinergics': set(),
 'GLP1 mimetics': {'albiglutide',
  'dulaglutide',
  'exenatide',
  'liraglutide',
  'lixisenatide',
  'semaglutide'},
 "Sulfonylureas (couldn't find a drug advised against, so found a drug worked well with) ": set(),
 'Iodinated contrast media': set()}

In [56]:
df_new[df_new['Source'] == 'chlorpropamide']

,Class,Source,Alternate Names,Adverse Class,Single Drug,Classification of Risk,Mechanism,Adverse Effect,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Drugs in Adverse Class
17,Sulfonylureas,chlorpropamide,NaN,NaN,Fluconazole,Moderate,Inhibition of CYP2C9,Elevated risk of hypoglycemia,NaN,NaN,NaN,NaN,NaN
18,Sulfonylureas,chlorpropamide,NaN,NaN,Miconazole,Moderate,Inhibition of CYP2C9,Elevated risk of hypoglycemia,NaN,NaN,NaN,NaN,NaN
35,Sulfonylureas,chlorpropamide,NaN,Fibrates,NaN,Moderate,Inhibition of CYP2C9,Elevated risk of hypoglycemia,NaN,NaN,NaN,NaN,NaN
36,Sulfonylureas,chlorpropamide,NaN,H2-antagonoists,NaN,Moderate,Inhibition of CYP2C9,Elevated risk of hypoglycemia,NaN,NaN,NaN,NaN,NaN


In [46]:
df_classes_slice[df_classes_slice['Source'] == 'nateglinide']

,Source,Drugs in Adverse Class,Classification of Risk
15,nateglinide,linagliptin,NaN
15,nateglinide,alogliptin,NaN
15,nateglinide,sitagliptin,NaN
15,nateglinide,saxagliptin,NaN
53,nateglinide,albiglutide,Moderate
53,nateglinide,semaglutide,Moderate
53,nateglinide,lixisenatide,Moderate
53,nateglinide,dulaglutide,Moderate
53,nateglinide,exenatide,Moderate
53,nateglinide,liraglutide,Moderate
